In [2]:
from models import scalers, decompositions, estimators, create_params_grid
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from imblearn.metrics import classification_report_imbalanced
from imblearn.pipeline import Pipeline
import pandas as pd
import neptune.new as neptune

In [4]:
run = neptune.init(project='projektmlcdv/ml-project-MW-MM', api_token = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIwYmI4NjVmYS0zM2JlLTRkZTQtOTI5YS0zNjY4NzhjNGQ4YjYifQ==')

https://app.neptune.ai/projektmlcdv/ml-project-MW-MM/e/MLPROJ-34
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [3]:
decompositions = {key: decompositions[key] for key in ['pca']}
scalers = { key: scalers[key] for key in ['StandardScaler']}
estimators = { key: estimators[key] for key in [ 'SVC'] }

params_grid = create_params_grid(scalers, decompositions, estimators)
params_grid[0]

{'scaler': [StandardScaler()],
 'decomposition': [PCA()],
 'decomposition__n_components': [2, 5, 10, 100, 0.95, 0.7],
 'decomposition__whiten': [False, True],
 'decomposition__svd_solver': ['auto', 'full', 'arpack', 'randomized'],
 'estimator': [SVC()],
 'estimator__C': array([ 10.        ,  12.91549665,  16.68100537,  21.5443469 ,
         27.82559402,  35.93813664,  46.41588834,  59.94842503,
         77.42636827, 100.        ]),
 'estimator__class_weight': ['balanced'],
 'estimator__kernel': ['linear', 'rbf', 'poly'],
 'estimator__gamma': ['scale', 'auto']}

#### Creating Pipeline

In [6]:
pipe = Pipeline(steps=[('scaler', StandardScaler()),('decomposition',PCA()),('estimator', SVC())])

#### GridSearch

In [7]:
grid = GridSearchCV(pipe, params_grid,verbose = 2, cv=4, n_jobs = -1, scoring = ['f1_weighted','recall_weighted','precision_weighted'],refit = False,)

In [8]:
encoder = lambda x:1 if x == -1 else 0
decoder = lambda x:-1 if x == 1 else 1

In [9]:
y = pd.read_csv('train_labels.csv', header=None, names=['y'])
X = pd.read_csv("train_data.csv",header=None, low_memory = False)

In [10]:
y= y['y'].apply(encoder)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42)

In [12]:
best = grid.fit(X_train,y_train)

Fitting 4 folds for each of 2880 candidates, totalling 11520 fits


KeyboardInterrupt: 

In [34]:
run["cv_results"].upload(neptune.types.File.as_html(pd.DataFrame.from_dict(best.cv_results_)))

Run MLPROJ-30 received stop signal. Exiting


Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 3 operations to synchronize with Neptune. Do not kill this process.


All 3 operations synced, thanks for waiting!


In [32]:
best

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('decomposition', PCA()),
                                       ('estimator', SVC())]),
             n_jobs=-1,
             param_grid=[{'decomposition': [PCA()],
                          'decomposition__n_components': [2, 5, 10, 100, 0.95,
                                                          0.7],
                          'decomposition__svd_solver': ['auto', 'full',
                                                        'arpack',
                                                        'randomized'],
                          'decomposition__whiten': [False, True],
                          'estimator': [SVC()],
                          'estimator__C': array([ 10.        ,  12.91549665,  16.68100537,  21.5443469 ,
        27.82559402,  35.93813664,  46.41588834,  59.94842503,
        77.42636827, 100.        ]),
                          'estimator_